In [8]:
import numpy as np
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep

In [50]:
# vytvoříme si slovník s kraji a jejich labely, které jim web přiřadil
link = f'https://www.in-pocasi.cz/archiv/archiv.php?historie=2021-10-08&region=1'
raw = requests.get(link).content
soup = BeautifulSoup(raw, 'html.parser')
labels = []
names = []
for region in soup.find("select", {"name": "archiv_kraj"}).findAll("option"):
    labels.append(region["value"])

    if region.text == "Praha":
        name = "Praha"
    elif region.text == "Vysočina":
        name = "Vysočina"
    else: name = region.text + " kraj"
    names.append(name)
regions = dict(zip(labels, names))

In [115]:
# web obsahuje dva typy meteor. stanic a každý má trochu jiné měřené hodnoty, proto ze začátku vytvoříme 2 datasety
list_klimaticke_stanice = []
list_soukrome_stanice = []
date = "2021-10-08"
for region_label in list(regions.keys()): # projdeme všechny kraje

    link = f'https://www.in-pocasi.cz/archiv/archiv.php?historie={date}&region={region_label}'
    raw = requests.get(link).content
    soup = BeautifulSoup(raw, 'html.parser') # uložíme zdrojový kód
    soup_tables = soup.find('div',{'class':'typography'}).findAll("tbody") # tabulky s daty

    for type, table in enumerate(soup_tables):
        soup_rows = table.findAll("tr")

        for row in soup_rows:
            data = [] # list na data z konkrétní stanice
            
            for element in row.findAll("td"): # přiřazujeme všechny hodnoty
                data.append(element.text)
            data.append(regions[region_label]) # přiřadíme kraj
            data.append(date) #přiřadíme datum
            data.append(row.findAll("td")[0].find("a")["href"]) # uložíme si odkaz na meteor. stanici

            if type == 0: # přidáme do příslušného listu
                list_klimaticke_stanice.append(data)
            else: list_soukrome_stanice.append(data)

# vytvoříme datasety s neupravenými daty (pouze stringy)
cols_klima = ["Stanice", "Maximální teplota (°C)", "Minimální teplota (°C)", "Náraz větru (km/h)", "Srážky (mm)", "Sněhová pokrývka (cm)", "Sluneční svit (hod)", "Kraj", "Datum", "Odkaz"]
cols_soukr = ["Stanice", "Maximální teplota (°C)", "Minimální teplota (°C)", "Náraz větru (km/h)", "Srážky (mm)", "Nejvyšší tlak (hPa)", "Nejvyšší vlhkost (%)", "Kraj", "Datum", "Odkaz"]
raw_df_klima = pd.DataFrame(list_klimaticke_stanice, columns=cols_klima)
raw_df_soukr = pd.DataFrame(list_soukrome_stanice, columns=cols_soukr)

# převedeme data na čísla
def substitute(string):
    parts = string.split(" ")
    if len(parts) == 1:
        return np.nan
    elif len(parts) == 2:
        try:
            return float(parts[0])
        except ValueError as err:
            print("Inappropriate format: ", err) 
    else: raise ValueError("Inappropriate format!")

def modify_values(init_df):
    df = init_df.copy()
    for col in df.columns[1:7]:
        df[col] = df[col].apply(substitute)
    return df

df_klima = modify_values(raw_df_klima)
df_soukr = modify_values(raw_df_soukr)

In [204]:
# ciselnik stazen z https://www.czso.cz
ciselnik = pd.read_csv('data/ciselnik_uzemi_CR_1_1_2021.csv')
df = ciselnik.iloc[:, :2] # začátek datasetu

# data z https://www.rozpocetobce.cz
coords = pd.read_csv('data/coords.csv')

# nalezneme chybné řádky a smažeme:
# for i, mun in enumerate(coords['OBEC_CSU_KOD']):
#     if len(mun) != 6: 
#         print(f"row {i} has invalid length!")
coords = coords.drop(index= 5835)
# změníme datový typ
coords = coords.astype({'OBEC_CSU_KOD': int})
# přidáme data do datasetu
df = df.merge(coords, how="outer", left_on= "kod_obec", right_on= "OBEC_CSU_KOD").drop(["OBEC_NAZEV", "OBEC_CSU_KOD"], axis=1)

# vyscrapujeme potřebná data z wikipedie (musíme ručně poskytnout odkazy)
missing_id = list(df[df["LONGITUDE"].isna()]["kod_obec"])
missing_data = []
links = ['https://cs.wikipedia.org/wiki/Město_Libavá', 'https://cs.wikipedia.org/wiki/Bražec_(okres_Karlovy_Vary)', 
'https://cs.wikipedia.org/wiki/Vojenský_újezd_Boletice', 'https://cs.wikipedia.org/wiki/Vojenský_újezd_Březina',
'https://cs.wikipedia.org/wiki/Doupovské_Hradiště', 'https://cs.wikipedia.org/wiki/Hlučín',
'https://cs.wikipedia.org/wiki/Vojenský_újezd_Hradiště', 'https://cs.wikipedia.org/wiki/Kozlov_(okres_Olomouc)',
'https://cs.wikipedia.org/wiki/Krhová', 'https://cs.wikipedia.org/wiki/Vojenský_újezd_Libavá',
'https://cs.wikipedia.org/wiki/Libhošť', 'https://cs.wikipedia.org/wiki/Luboměř_pod_Strážnou',
'https://cs.wikipedia.org/wiki/Město_Libavá', 'https://cs.wikipedia.org/wiki/Petrov_nad_Desnou',
'https://cs.wikipedia.org/wiki/Poličná', 'https://cs.wikipedia.org/wiki/Polná_na_Šumavě',
'https://cs.wikipedia.org/wiki/Želešice']
for l in links:
    source = requests.get(l).content
    soup_obce = BeautifulSoup(source, 'html.parser')
    lat_string = soup_obce.find('span', {'class': 'coordinates'}).find_all('span')[0].text
    lon_string = soup_obce.find('span', {'class': 'coordinates'}).find_all('span')[1].text
    
    try: 
        lat = float(lat_string[:lat_string.find('°')]) + float(lat_string[lat_string.find('°') + 1:lat_string.find('′')]) / 60 + float(lat_string[lat_string.find('′') + 1: lat_string.find('″')]) / 3600
    except:
         lat = float(lat_string[:lat_string.find('°')]) + float(lat_string[lat_string.find('°') + 1:lat_string.find('′')]) / 60
    
    try: 
        lon = float(lon_string[:lon_string.find('°')]) + float(lon_string[lon_string.find('°') + 1:lon_string.find('′')]) / 60 + float(lon_string[lon_string.find('′') + 1: lon_string.find('″')]) / 3600
    except:
        lon = float(lon_string[:lon_string.find('°')]) + float(lon_string[lon_string.find('°') + 1:lon_string.find('′')]) / 60 
    
    try:
        psc = ''.join((soup_obce.find('th', string = "PSČ").find_next('td').text).split(' '))
    except: psc = np.nan
    adresa = str(soup_obce.find('th', string = "Kontakt").find_next('td')).split('<br/>')[0][4:]
    ico = np.nan
    try:
        email = soup_obce.find('th', string = "Kontakt").find_next('td').find('a')['href'][7:]
    except: 
        email = np.nan
    missing_data.append([adresa, psc, ico, l, lat, lon, email])

# doplníme chybějící hodnoty
for i,id in enumerate(missing_id):
    for j in range(7):
        df.loc[df["kod_obec"] == id, df.columns[3 + j]] = missing_data[i][j]

In [205]:
df

,kod_obec,nazev_obec,OBEC_ORG_NAZEV,OBEC_ORG_ULICE,OBEC_ORG_PSC,OBEC_ICO_ORG,URL_WIKIPEDIA,LONGITUDE,LATITUDE,EMAIL_PODATELNA
0,554979,Abertamy,OÚ Abertamy,Farní 2,36235,254398,http://cs.wikipedia.org/wiki/Abertamy,12.818260,50.368745,ou@obecabertamy.cz
1,531367,Adamov,OÚ Adamov,Zbýšov 62,28601,639664,http://cs.wikipedia.org/wiki/Adamov_(okres_Kut...,15.408913,49.857817,podatelna@obecadamov.cz
2,535826,Adamov,OÚ Adamov,V chalupách 47,37371,581160,http://cs.wikipedia.org/wiki/Adamov_(okres_%C4...,14.539503,49.000520,info@adamovcb.eu
3,581291,Adamov,MěÚ Adamov,Pod Horkou 101/2,67904,279889,http://cs.wikipedia.org/wiki/Adamov_(okres_Bla...,16.658550,49.300501,mesto@adamov.cz
4,547786,Adršpach,OÚ Adršpach,Horní Adršpach 128,54952,653560,http://cs.wikipedia.org/wiki/Adr%C5%A1pach,16.108568,50.618659,ou.adrspach@seznam.cz
...,...,...,...,...,...,...,...,...,...,...
6253,555762,Žlutice,MěÚ Žlutice,Velké nám.144,36452,255181,http://cs.wikipedia.org/wiki/%C5%BDlutice,13.162971,50.091920,meu@zlutice.cz
6254,541575,Žulová,OÚ Žulová,Hlavní 36,79065,303682,http://cs.wikipedia.org/wiki/%C5%BDulov%C3%A1,17.098709,50.309335,ou.zulova@jen.cz
6255,572641,Žumberk,OÚ Žumberk,NaN,53836,271292,http://cs.wikipedia.org/wiki/%C5%BDumberk,15.857795,49.874323,zumberk.ou@worldonline.cz
6256,562424,Županovice,OÚ Županovice,NaN,37881,666599,http://cs.wikipedia.org/wiki/%C5%BDupanovice_(...,15.506123,48.957110,obec.zupanovice@seznam.cz


In [6]:
from geopy.distance import great_circle
coords_1 = (49.9514797, 16.1577783)
coords_2 = (49.9183972, 16.0693678)

print(great_circle(coords_1, coords_2).km)

7.319262734964349
